In [1]:
# 라이브러리 로드
# requests는 작은 웹브라우저로 웹사이트 내용을 가져온다.
import requests
# BeautifulSoup 을 통해 읽어 온 웹페이지를 파싱한다.
from bs4 import BeautifulSoup as bs
# 크롤링 후 결과를 데이터프레임 형태로 보기 위해 불러온다.
import pandas as pd

In [2]:
# 크롤링 할 사이트
kospi_url = "https://finance.naver.com/sise/sise_low_up.nhn?sosok=0"
kosdak_url = "https://finance.naver.com/sise/sise_low_up.nhn?sosok=1"
response = requests.get( kospi_url )
response

<Response [200]>

In [ ]:
# 라이브러리 로드
# requests는 작은 웹브라우저로 웹사이트 내용을 가져온다.
import requests
# BeautifulSoup 을 통해 읽어 온 웹페이지를 파싱한다.
from bs4 import BeautifulSoup as bs
# 크롤링 후 결과를 데이터프레임 형태로 보기 위해 불러온다.
import pandas as pd

kospi_url = "https://finance.naver.com/sise/sise_low_up.nhn?sosok=0"
kosdak_url = "https://finance.naver.com/sise/sise_low_up.nhn?sosok=1"
list_url = [
    kospi_url, kosdak_url
]

def remake_list(list_base):
    num_cols = [3,5,7,8,9,10]
    rate_cols = [0,6]
    for idx in range(len(num_cols)):
        try:
            num = int(list_base[num_cols[idx]].replace(",",""))
        except:
            num = None
        list_base[num_cols[idx]] = num
    for idx in range(len(rate_cols)):
        try:
            num = float(list_base[rate_cols[idx]].replace(",","").replace("%",""))
        except:
            num = None
        list_base[rate_cols[idx]] = num
        
    return list_base


list_cols = [
    "저가 등락률","종목코드","종목명","현재가","부호","전일비","전일 등락률","시가","고가","저가","거래량","PER","ROE"
]        
list_whole = []

def get_data(base_url):
    response = requests.get( base_url )
    soup = bs(response.text, 'html.parser')

    content = soup.select("div.box_type_l")
    list_content = str(content).split("\n")
    list_jongmok = []
    for row_data in list_content:
        row = row_data.strip().replace("\t","")
        if '<td class="no">' in row:
            if len(list_jongmok) > 0:
                list_whole.append(list_jongmok)
                list_jongmok = []
        elif "tltle" in row:
            jongmok_cd = row.split("code=")[1].split('"')[0]
            jongmok_nm = row.split("code=")[1].split('"')[1].split('</a')[0].replace(">","")
            list_jongmok.append("A" + jongmok_cd)
            list_jongmok.append(jongmok_nm)
        elif "<img" in row:
            if "하락" in row:
                sign = -1
            else:
                sign = 1
            list_jongmok.append(sign)
        elif row[:1] != "<":
            list_jongmok.append(row)
        elif ("<td" not in row or row[:4] == "<tr>"):
            continue
        else:
            num_val = row.replace('<td class="number">','')
            if len(num_val) == 0:
                continue
            list_jongmok.append(num_val.replace("</td>",""))


for url in list_url:
    get_data(url)
    
list_df = []
for list_row in list_whole:
    if len(list_row) == 13:
        list_df.append(remake_list(list_row))
    
df_rising = pd.DataFrame(list_df, columns=list_cols)
df_rising = df_rising.sort_values(["저가 등락률"], ascending=False)
df_rising = df_rising.loc[df_rising["저가 등락률"] > 10.0]
df_rising = df_rising.loc[df_rising["전일 등락률"] > 10.0]
df_rising = df_rising.loc[df_rising["거래량"] > 1000000]
df_rising

In [1]:
import requests
from bs4 import BeautifulSoup as bs

kospi_url = "https://finance.naver.com/sise/sise_low_up.nhn?sosok=0"
kosdak_url = "https://finance.naver.com/sise/sise_low_up.nhn?sosok=1"
list_url = [
    kospi_url, kosdak_url
]

def remake_list(list_base):
    num_cols = [3,4,6,7,8,9]
    rate_cols = [0,5,10,11]
    for idx in range(len(num_cols)):
        try:
            num = int(list_base[num_cols[idx]].replace(",",""))
        except:
            num = 0
        list_base[num_cols[idx]] = num
    for idx in range(len(rate_cols)):
        try:
            num = float(list_base[rate_cols[idx]].replace(",","").replace("%",""))
        except:
            num = 0
        list_base[rate_cols[idx]] = num
        
    return list_base


list_cols = [
    "저가 등락률","종목코드","종목명","현재가","전일비","전일 등락률","시가","고가","저가","거래량","PER","ROE"
]        
list_whole = []

def get_data(base_url):
    response = requests.get( base_url )
    soup = bs(response.text, 'html.parser')

    content = soup.select("div.box_type_l")
    list_content = str(content).split("\n")
    list_jongmok = []
    for row_data in list_content:
        row = row_data.strip().replace("\t","")
        if '<td class="no">' in row:
            if len(list_jongmok) > 0:
                list_whole.append(list_jongmok)
                list_jongmok = []
        elif "tltle" in row:
            jongmok_cd = row.split("code=")[1].split('"')[0]
            jongmok_nm = row.split("code=")[1].split('"')[1].split('</a')[0].replace(">","")
            list_jongmok.append("A" + jongmok_cd)
            list_jongmok.append(jongmok_nm)
        elif row[:1] != "<":
            list_jongmok.append(row)
        elif ("<td" not in row or row[:4] == "<tr>"):
            continue
        else:
            num_val = row.replace('<td class="number">','')
            if len(num_val) == 0:
                continue
            list_jongmok.append(num_val.replace("</td>",""))


for url in list_url:
    get_data(url)
    
list_result = []
for list_row in list_whole:
    if len(list_row) == 12:
        list_result.append(remake_list(list_row))
        
head = """
insert into naver_low_vs_rt
(low_vs_rt, jongmok_cd, jongmok_nm, prc, pre_vs_gap, pre_vs_rt, start_prc, high_prc, low_prc, vol, per, roe)
values
"""
body = ""
for list_row in list_result:
    row = "("
    for idx in range(len(list_row)):
        try:
            row += "'" + list_row[idx] + "', "
        except:
            row += str(list_row[idx]) + ", "
    row += ")," + "\n"
    row = row.replace(", )", ")")
    body += row

ins_qry = head + body
print(ins_qry[:len(ins_qry)-2])


insert into naver_low_vs_rt
(low_vs_rt, jongmok_cd, jongmok_nm, prc, pre_vs_gap, pre_vs_rt, start_prc, high_prc, low_prc, vol, per, roe)
values
(31.65, 'A011000', '진원생명과학', 29950, 6900, 29.93, 23350, 29950, 22750, 37551217, -68.54, -30.43),
(17.39, 'A007110', '일신석재', 2700, 395, 17.14, 2335, 2825, 2300, 114822689, 180.0, 1.22),
(16.67, 'A006345', '대원전선우', 7000, 520, 8.02, 6000, 7000, 6000, 506903, 170.73, 0),
(14.42, 'A078520', '에이블씨엔씨', 12300, 1250, 11.31, 10900, 12500, 10750, 1115706, -4.0, -43.75),
(11.64, 'A009275', '신원우', 58500, 6100, 11.64, 52400, 64900, 52400, 22565, -1950.0, 0),
(10.4, 'A030790', '비케이탑스', 9660, 670, 7.45, 9000, 9840, 8750, 459975, -2.73, 0),
(9.1, 'A007660', '이수페타시스', 4615, 330, 7.7, 4285, 4615, 4230, 2648362, -20.51, -19.14),
(8.92, 'A002140', '고려산업', 4760, 355, 8.06, 4415, 4800, 4370, 3461390, 14.78, 6.91),
(8.76, 'A010600', '웰바이오텍', 3475, 150, 4.51, 3280, 3580, 3195, 2831055, -2.3, -141.53),
(8.4, 'A001530', 'DI동일', 200000, 12500, 6.67, 187500, 202000, 18450